# Installing COLMAP on Google Colab

COLMAP is a popular 3D reconstruction software. It takes multiple images of a scene and reconstructs the 3D structure of that scene along with the camera positions. It doesn Feature Detection & Matching, Structure-from-Motion (SfM), and Multi-View Stereo (MVS).

## Install Dependencies

In [ ]:
!sudo apt update
!sudo apt install -y \
    build-essential cmake git \
    libboost-all-dev libeigen3-dev libsuitesparse-dev \
    qtbase5-dev libglew-dev libglfw3-dev \
    libx11-dev libopencv-dev libgoogle-glog-dev \
    libgflags-dev libatlas-base-dev libopencv-core-dev \
    libopenimageio-dev openimageio-tools libopenexr-dev \
    libcgal-dev libcgal-qt5-dev libmetis-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,860 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,966 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,633 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,572 kB]
Get:14 http://archive

## Install Abseil
Abseil is a collection of open-source C++ libraries from Google. Ceres depends on Abseil for some of its internal utilities like containers, flags, or threading helpers.

In [ ]:
%cd /content
!git clone https://github.com/abseil/abseil-cpp.git
%cd /content/abseil-cpp
!git checkout 20230802.0

!rm -rf build
!mkdir build
%cd build
!cmake .. \
  -DCMAKE_BUILD_TYPE=Release \
  -DCMAKE_INSTALL_PREFIX=/usr/local \
  -DCMAKE_POSITION_INDEPENDENT_CODE=ON
!make -j$(nproc)
!sudo make install

/content
Cloning into 'abseil-cpp'...
remote: Enumerating objects: 33746, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 33746 (delta 87), reused 56 (delta 56), pack-reused 33584 (from 3)
Receiving objects: 100% (33746/33746), 19.19 MiB | 14.91 MiB/s, done.
Resolving deltas: 100% (26422/26422), done.
/content/abseil-cpp
Note: switching to '20230802.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 29bf8085 Cherry pick CCTZ fix for MinGW (

## Install Ceres Solver using the installed Abseil
Ceres is an open-source C++ library for non-linear least squares optimization. COLMAP uses Ceres to optimize camera poses, 3D points, and bundle adjustment.

In [ ]:
%cd /content
!git clone https://github.com/ceres-solver/ceres-solver.git
%cd /content/ceres-solver
!git checkout 2.1.0

!rm -rf build
!mkdir build
%cd build
!cmake .. \
  -DBUILD_TESTING=OFF \
  -DBUILD_EXAMPLES=OFF \
  -DCMAKE_BUILD_TYPE=Release \
  -DCMAKE_INSTALL_PREFIX=/usr/local
!make -j$(nproc)
!sudo make install

/content
Cloning into 'ceres-solver'...
remote: Enumerating objects: 23212, done.
remote: Counting objects: 100% (585/585), done.
remote: Compressing objects: 100% (370/370), done.
remote: Total 23212 (delta 343), reused 218 (delta 212), pack-reused 22627 (from 3)
Receiving objects: 100% (23212/23212), 18.14 MiB | 16.88 MiB/s, done.
Resolving deltas: 100% (16126/16126), done.
/content/ceres-solver
Note: switching to '2.1.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at f68321e7 Update version history
/conten

## Install COLMAP (CPU-only)

In [ ]:
%cd /content
!git clone https://github.com/colmap/colmap.git
%cd colmap

!rm -rf build
!mkdir build
%cd build
!cmake .. \
    -DCMAKE_BUILD_TYPE=Release \
    -DCMAKE_INSTALL_PREFIX=/usr/local \
    -DCUDA_ENABLED=OFF \
    -DCeres_DIR=/usr/local/lib/cmake/Ceres \
    -DAbsl_DIR=/usr/local/lib/cmake/absl

!make -j$(nproc)
!sudo make install

/content
Cloning into 'colmap'...
remote: Enumerating objects: 32439, done.
remote: Counting objects: 100% (611/611), done.
remote: Compressing objects: 100% (450/450), done.
remote: Total 32439 (delta 407), reused 167 (delta 161), pack-reused 31828 (from 4)
Receiving objects: 100% (32439/32439), 73.28 MiB | 21.42 MiB/s, done.
Resolving deltas: 100% (24777/24777), done.
/content/colmap
/content/colmap/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Enabling LSD support
-- Found OpenMP_C: -fopenmp (found version "4.5")
-- Found O

### Test COLMAP installation

In [ ]:
!colmap --help

COLMAP 3.14.0.dev0 -- Structure-from-Motion and Multi-View Stereo
(Commit 060f75e9 on 2025-12-22 without CUDA)

Usage:
  colmap [command] [options]
Documentation:
  https://colmap.github.io/
Example usage:
  colmap help [ -h, --help ]
  colmap gui
  colmap gui -h [ --help ]
  colmap automatic_reconstructor -h [ --help ]
  colmap automatic_reconstructor --image_path IMAGES --workspace_path WORKSPACE
  colmap feature_extractor --image_path IMAGES --database_path DATABASE
  colmap exhaustive_matcher --database_path DATABASE
  colmap mapper --image_path IMAGES --database_path DATABASE --output_path MODEL
  ...
Available commands:
  help
  gui
  automatic_reconstructor
  bundle_adjuster
  color_extractor
  database_cleaner
  database_creator
  database_merger
  delaunay_mesher
  exhaustive_matcher
  feature_extractor
  feature_importer
  geometric_verifier
  global_mapper
  global_mapper_resume
  guided_geometric_verifier
  hierarchical_mapper
  image_deleter
  image_filterer
  image_rectif

### Feature extraction using COLMAP (CPU)
Folder structure

```
/content/project/
├── images/        # your input images
├── database.db
└── sparse/

```

You can put images in your Google Drive, ```image_source``` folder, and change the path below. This command will copy images from the ```image_source``` folder into your Colab space and work on it.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content
!rm -rf project
!mkdir -p project/images
!cp -r "/content/drive/MyDrive/image_source/." "/content/project/images"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content


### Start Feature Extraction

If your images are from different cameras, set ```ImageReader.single_camera``` to ```0```, else set to ```1```.

In [ ]:
!export QT_QPA_PLATFORM=offscreen
!colmap feature_extractor \
  --database_path /content/project/database.db \
  --image_path /content/project/images \
  --ImageReader.single_camera 0 \
  --FeatureExtraction.use_gpu 0

W1223 15:42:48.182210 33985 feature_extraction.cc:416] Your current options use the maximum number of threads on the machine to extract features. Extracting SIFT features on the CPU can consume a lot of RAM per thread for large images. Consider reducing the maximum image size and/or the first octave or manually limit the number of extraction threads. Ignore this warning, if your machine has sufficient memory for the current settings.
I1223 15:42:48.182617 33996 misc.cc:44] 
Feature extraction
I1223 15:42:48.183372 33999 sift.cc:725] Creating SIFT CPU feature extractor
I1223 15:42:48.183424 34000 sift.cc:725] Creating SIFT CPU feature extractor
I1223 15:47:24.327934 34001 feature_extraction.cc:262] Processed file [1/28]
I1223 15:47:24.329080 34001 feature_extraction.cc:265]   Name:            1795f43eca7211f0a922027c7c2f8ab1.tif
I1223 15:47:24.329103 34001 feature_extraction.cc:274]   Dimensions:      18432 x 12288
I1223 15:47:24.329114 34001 feature_extraction.cc:277]   Camera:        

### Feature matching (exhaustive)

In [ ]:
!colmap exhaustive_matcher \
  --database_path /content/project/database.db \
  --FeatureMatching.use_gpu 0

### Sparse reconstruction

In [ ]:
!mkdir -p /content/project/sparse
!colmap mapper \
  --database_path /content/project/database.db \
  --image_path /content/project/images \
  --output_path /content/project/sparse

### Export undistorted COLMAP reconstruction

In [ ]:
!colmap image_undistorter \
    --image_path /content/project/images \
    --input_path /content/project/sparse/0 \
    --output_path /content/project/undistorted \
    --output_type COLMAP \
    --max_image_size 1600

### Copy COLMAP results to Google Drive

In [ ]:
!mkdir "/content/drive/MyDrive/COLMAP Results"
!cp -r "/content/project" "/content/drive/MyDrive/COLMAP Results"